In [1]:
import pandas as pd

data_path = "../DataPreProcessing/.out/license_plates.csv"
image_path = "../OCR/data/LP-characters/images"

data = pd.read_csv(data_path, dtype=str)

In [2]:
import PIL as pil
from PIL import Image
from skimage.feature import hog
import matplotlib.pyplot as plt
import cv2

def prepare_img(img, xmin, ymin, xmax, ymax):
        # crop image to bounding box
        char_img = img[ymin:ymax, xmin:xmax]

        # scale image
        char_img = cv2.resize(char_img, (64, 64))
        #plt.figure()
        hog_img = hog(char_img, orientations=9, pixels_per_cell=(4,4), cells_per_block=(1, 1))

        return hog_img

def grab_bbs(row):
    img = cv2.imread(f"{image_path}/{row['filename'].split('.')[0]}.png", 0)

    bbs = []
    charcoords = row[2:]
    charcoords = charcoords[charcoords.notnull()]
    
    for i in range(0, len(charcoords), 2):
        char = charcoords[i]
        coords = charcoords[i+1]

        bbs.append((char, *map(int, coords.split("-"))))

    return bbs

In [3]:
# set all non-nan characters to 1
for i, row in data.iterrows():
    chars = row.filter(like="char")
    chars[chars.notnull()] = "letter"
    data.loc[i, chars.index] = chars

In [4]:
listclasses = [x for x in pd.unique(data.filter(like='char').values.flatten()) if str(x) != 'nan']
class2id = {c: i for i, c in enumerate(listclasses)}

def to_yolo_format(row):
    img = cv2.imread(f"{image_path}/{row['filename'].split('.')[0]}.png", 0)
    img_width, img_height = img.shape[::-1]

    bbs = grab_bbs(row)

    yolo_format = []
    for bb in bbs:
        char, xmin, ymin, xmax, ymax = bb

        width = (xmax - xmin) / img_width
        height = (ymax - ymin) / img_height
        
        x_center = ((xmin + xmax) / 2) / img_width
        y_center = ((ymin + ymax) / 2) / img_height
        

        yolo_format.append(f"{class2id[char]} {x_center} {y_center} {width} {height}")

    return "\n".join(yolo_format)

In [5]:
import os
cwd = os.getcwd().replace("\\", "/")

In [6]:
data_dir = "data/.out/yolo_data"
train_dir = f"{data_dir}/images/train"
test_dir = f"{data_dir}/images/test"
lbl_train_dir = f"{data_dir}/labels/train"
lbl_test_dir = f"{data_dir}/labels/test"


_split = 0.8

i = 0
for _, row in data.iterrows():
    i += 1
    base_name = row['filename'].split('.')[0]

    if i < len(data) * _split:
        with open(f"{lbl_train_dir}/{base_name}.txt", "w") as f:
            f.write(to_yolo_format(row) + "\n")
            # copy image into train folder
        img = cv2.imread(f"{image_path}/{row['filename'].split('.')[0]}.png")
        cv2.imwrite(f"{train_dir}/{row['filename'].split('.')[0]}.png", img)
    else:
        with open(f"{lbl_test_dir}/{base_name}.txt", "w") as f:
            f.write(to_yolo_format(row) + "\n")
            # copy image into test folder
        img = cv2.imread(f"{image_path}/{row['filename'].split('.')[0]}.png")
        cv2.imwrite(f"{test_dir}/{row['filename'].split('.')[0]}.png", img)
    

# create data.yaml
name_lst = [f' {class2id[c]}: {c}' for c in listclasses]
data_yaml = f"""
path: "{cwd}/{data_dir}"
train: "images/train"
val: "images/test"
nc: {len(listclasses)}
names: \n""" + ("\n".join(name_lst))


with open(f"{data_dir}/data.yaml", "w") as f:
    f.write(data_yaml)

## Training

In [7]:
from ultralytics import YOLO
model = YOLO("yolov8n.pt")  # load a pre-trained model (recommended for training)

results = model.train(data = data_dir+"/data.yaml", epochs=100, batch = 4, close_mosaic = False)

New https://pypi.org/project/ultralytics/8.2.2 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.15 🚀 Python-3.11.4 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=data/.out/yolo_data/data.yaml, epochs=100, time=None, patience=50, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train30, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=False, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, e

train: Scanning B:\projects\uconn\CSE5717\group_proj\CSE5717-Project\OCR\data\.out\yolo_data\labels\train.cache... 167 images, 0 backgrounds, 0 corrupt: 100%|██████████| 167/167 [00:00<?, ?it/s]
val: Scanning B:\projects\uconn\CSE5717\group_proj\CSE5717-Project\OCR\data\.out\yolo_data\labels\test.cache... 42 images, 0 backgrounds, 0 corrupt: 100%|██████████| 42/42 [00:00<?, ?it/s]


Plotting labels to runs\detect\train30\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train30
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100     0.868G      1.185      2.028      1.063         84        640: 100%|██████████| 42/42 [00:04<00:00,  9.70it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:00<00:00,  6.23it/s]

                   all         42        413      0.942      0.913      0.961       0.65


...

    100/100     0.847G     0.7252      0.359     0.8867         68        640: 100%|██████████| 42/42 [00:04<00:00, 10.18it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:00<00:00, 17.27it/s]

                   all         42        413      0.993      0.995      0.994      0.761



100 epochs completed in 0.136 hours.
Optimizer stripped from runs\detect\train30\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train30\weights\best.pt, 6.2MB

Validating runs\detect\train30\weights\best.pt...
Ultralytics YOLOv8.1.15 🚀 Python-3.11.4 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:00<00:00,  9.08it/s]


                   all         42        413       0.99       0.99      0.994       0.77
Speed: 0.3ms preprocess, 6.6ms inference, 0.0ms loss, 3.1ms postprocess per image
Results saved to runs\detect\train30


In [20]:
img = f"{test_dir}/0207.png"

pred_res = mod2(img)
img_width, img_height = cv2.imread(img, 0).shape[::-1]
# print bounding boxes
xyxys = pred_res[0].boxes.xyxy

# plot image
img = cv2.imread(img)
for xyxy in xyxys:
    xmin, ymin, xmax, ymax = map(int, xyxy)
    cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)

cv2.imshow("img", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


image 1/1 b:\projects\uconn\CSE5717\group_proj\CSE5717-Project\OCR\data\.out\yolo_data\images\test\0207.png: 192x640 12 letters, 33.6ms
Speed: 3.7ms preprocess, 33.6ms inference, 5.1ms postprocess per image at shape (1, 3, 192, 640)


In [18]:
import torch
torch.save(model.model, "model/yolo_char_rec.pt")

In [19]:
mod2 = YOLO("yolov8n.pt")
mod2.model = torch.load("model/yolo_char_rec.pt")

In [17]:
model.model.save("model/yolo_char_rec.pt")

TypeError: 'list' object is not callable